### Chat Models
A chat model is an LLM that:
- Accepts messages
- Returns messages

#### Why LangChain wraps LLMs?

Because in production:
- You want retries
- You want consistent interface
- You want to switch models easily

```
ChatGroq → LangChain wrapper

invoke() → single LLM call

.content → clean text output

```

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0
)

response = llm.invoke("What is LangChain?")
print(response.content)


**LangChain** is an open‑source framework that makes it easier to build applications powered by large language models (LLMs) such as OpenAI’s GPT‑4, Anthropic’s Claude, LLaMA, Gemini, and many others.  

---

## Core Idea

Instead of treating an LLM as a single “black‑box” that takes a prompt and returns text, LangChain encourages you to **compose** the LLM with other pieces of logic—data sources, memory, agents, tools, and control flow—so you can create more sophisticated, reliable, and interactive AI systems. Think of it as a “toolkit for chaining together” the building blocks that a real‑world LLM app needs.

---

## Main Building Blocks

| Block | What it does | Typical use‑case |
|-------|--------------|-----------------|
| **LLM Wrapper** | Uniform Python (or JS/TS) interface to any LLM provider (OpenAI, Azure, Cohere, HuggingFace, etc.) | Send prompts, get completions, stream tokens |
| **Prompt Templates** | Structured, reusable prompt strings with variable substitution, partia

### Prompts
A prompt is instructions for the LLM, not just a question.

Prompt = role + task + rules + format

#### Simple Prompt
Problem 

- Can’t reuse
- Can’t change dynamically

In [3]:
response = llm.invoke(
    "Explain Python to a beginner in simple English"
)
print(response.content)


## What is Python?

Python is a **programming language** – a set of words and rules that let you tell a computer what to do.  
Think of it like a very precise form of English that a computer can understand.

- **Easy to read:** Python code looks a lot like plain English, so it’s great for beginners.  
- **Very popular:** People use it for web sites, games, data analysis, artificial‑intelligence, automation, and more.  
- **Free and open‑source:** You can download it for free and use it on Windows, macOS, Linux, and even on phones or tiny devices like Raspberry Pi.

---

## How do you write and run Python?

1. **Install Python** – go to <https://python.org> and download the installer for your operating system.  
2. **Write code** – you can use a simple text editor (Notepad, TextEdit) or a special program called an *IDE* (e.g., VS Code, PyCharm, Thonny). Save your file with the extension `.py`, for example `hello.py`.  
3. **Run it** – open a terminal/command prompt, navigate to the fold

### PromptTemplate
Prompts are reused
- Inputs change
- Structure stays same

```
{topic} is a variable

Same prompt → many inputs
```

In [4]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["topic"],
    template="Explain {topic} to a beginner in simple English"
)

formatted_prompt = prompt.format(topic="LangChain")
response = llm.invoke(formatted_prompt)

print(response.content)


**What is LangChain?**  
Think of LangChain as a **tool‑box** that helps you build programs that can talk, think, and act using large language models (LLMs) like ChatGPT, Claude, or Llama. Instead of writing all the low‑level code yourself, LangChain gives you ready‑made building blocks that you can snap together—just like LEGO bricks.

---

## 1. Why do we need it?

- **LLMs are powerful, but they’re just one piece of a bigger puzzle.**  
  An LLM can generate text, but most real‑world apps also need to:
  - remember past conversations,
  - look up information from a database or the web,
  - do calculations,
  - follow a series of steps (a “chain”) to reach a result.

- **LangChain does the plumbing for you.**  
  It connects the LLM to other tools (APIs, memory, prompts, etc.) so you can focus on *what* you want the app to do, not *how* to wire everything together.

---

## 2. Core ideas (in plain English)

| Concept | What it means | Simple analogy |
|---------|---------------|-----

### Chains
A Chain connects Prompt + LLM.

#### What LangChain did:

1. Took input
2. Filled prompt
3. Called LLM
4. Returned output


In [5]:
from langchain.chains import LLMChain

chain = LLMChain(
    llm=llm,
    prompt=prompt
)

output = chain.run("LangChain")
print(output)


C:\Users\moiza\AppData\Local\Temp\ipykernel_2180\365076371.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(
C:\Users\moiza\AppData\Local\Temp\ipykernel_2180\365076371.py:8: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = chain.run("LangChain")


**What is LangChain?**  
Think of LangChain as a **tool‑box** that helps you build programs that can talk, think, and act using large language models (LLMs) like ChatGPT, Claude, or Llama. Instead of writing all the low‑level code yourself, LangChain gives you ready‑made building blocks that you can snap together—just like LEGO bricks.

---

## 1. Why do we need it?

- **LLMs are powerful, but they’re just one piece of a bigger puzzle.**  
  An LLM can generate text, but most real‑world apps also need to:
  - remember past conversations,
  - look up information from a database or the web,
  - do calculations,
  - follow a series of steps (a “chain”) to reach a result.

- **LangChain does the plumbing for you.**  
  It connects the LLM to other tools (APIs, memory, prompts, etc.) so you can focus on *what* you want the app to do, not *how* to wire everything together.

---

## 2. Core ideas (in plain English)

| Concept | What it means | Simple analogy |
|---------|---------------|-----

### Sequential Chains (MULTI-STEP THINKING)
Real problems need steps.

Example:
- Explain topic
- Summarize explanation

```Output of one chain → input of next```

In [7]:
from langchain.chains import SequentialChain, LLMChain
from langchain.prompts import PromptTemplate

# Prompt 1
explain_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Explain {topic} in detail"
)

# Prompt 2 (FIXED)
summary_prompt = PromptTemplate(
    input_variables=["explanation"],
    template="Summarize this text:\n{explanation}"
)

# Chain 1
explain_chain = LLMChain(
    llm=llm,
    prompt=explain_prompt,
    output_key="explanation"
)

# Chain 2
summary_chain = LLMChain(
    llm=llm,
    prompt=summary_prompt,
    output_key="summary"
)

# Sequential Chain
overall_chain = SequentialChain(
    chains=[explain_chain, summary_chain],
    input_variables=["topic"],
    output_variables=["explanation", "summary"],
    verbose=True
)

result = overall_chain({"topic": "LangChain"})
print(result)


C:\Users\moiza\AppData\Local\Temp\ipykernel_2180\63505215.py:38: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = overall_chain({"topic": "LangChain"})




> Entering new SequentialChain chain...

> Finished chain.
{'topic': 'LangChain', 'explanation': '## LangChain – A Deep‑Dive Overview  \n\nLangChain is an **open‑source framework** that makes it easier to build **applications powered by large language models (LLMs)**.  \nIt does this by providing a **modular, composable set of building blocks** that handle the repetitive, “glue‑code” tasks that arise when you want an LLM to:\n\n1. **Understand and generate text** (prompt‑engineering, LLM calls).  \n2. **Interact with external data or tools** (search, databases, APIs, code execution).  \n3. **Maintain state or memory** across turns.  \n4. **Be orchestrated in flexible pipelines** (chains, agents, agents‑with‑tools).  \n\nThink of LangChain as the **“standard library” for LLM‑centric software**, analogous to how Flask/Django are for web apps or Pandas for data manipulation.  \n\nBelow is a **structured, detailed guide** that covers:\n\n| Section | What you’ll learn |\n|---------|------

### Memory
LLMs forget everything.

- Memory = conversation state